# Занятие 6

#  Классификация задач машинного обучения. Метрики для оценки качества моделей. Линейная и логистическая регрессии.

Подзапросы. Табличные выражения. Оконные функции.


Одним из часто используемых и важных инструментов при написании SQL-запросов являются подзапросы.

**Подзапрос** — это запрос, использующийся в другом SQL запросе. Подзапрос всегда заключён в круглые скобки и обычно выполняется перед основным запросом.

Подзапрос может возвращать:

* одна строка и один столбец;

* нескольких строк с одним столбцом;

* нескольких строк с несколькими столбцами.

В зависимости от типа результирующего набора подзапроса определяются операторы, которые могут использоваться в основном запросе.

Подзапрос используется для получения данных, которые будут использованы основным запросом в качестве условия для фильтрации возвращаемых записей.

Подзапросы могут использоваться в инструкциях SELECT, INSERT, UPDATE и DELETE, а также с операторами =, <, >, >=, <=, IN, BETWEEN и т.д.



## Табличные выражения

**Обобщённое табличное выражение** или CTE (Common Table Expressions) - это временный результирующий набор данных, к которому можно обращаться в последующих запросах. Для написания обобщённого табличного выражения используется оператор WITH.

<code>
-- Пример использования конструкции WITH
WITH Aeroflot_trips AS
    (SELECT TRIP.* FROM Company
        INNER JOIN Trip ON Trip.company = Company.id WHERE name = "Aeroflot")

SELECT plane, COUNT(plane) AS amount FROM Aeroflot_trips GROUP BY plane;
</code>

Выражение с **WITH** считается «временным», потому что результат не сохраняется где-либо на постоянной основе в схеме базы данных, а действует как временное представление, которое существует только на время выполнения запроса, то есть оно доступно только во время выполнения операторов SELECT, INSERT, UPDATE, DELETE или MERGE. Оно действительно только в том запросе, которому он принадлежит, что позволяет улучшить структуру запроса, не загрязняя глобальное пространство имён.


## Оконные функции

Рассмотрим оконные функции на примере данных о конверсии по разным каналам привлечения клиентов.

Для справки:

* cpc  - контекстная реклама

* organic – бесплатная поисковая выдача

* dicrect - прямые переходы на сайт (пользователь в браузере вбил адрес сайта)

* cpa - модель оплаты рекламы, при которой учитывается стоимость за выполнение определенных действий на сайте.

In [1]:
from sqlalchemy import create_engine

In [2]:
engine = create_engine('sqlite:///SQL_s5.db', echo = False)

In [3]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [4]:
d = {'Date': ['18.03.2024', '18.03.2024','18.03.2024','19.03.2024','19.03.2024','19.03.2024','19.03.2024','19.03.2024','20.03.2024','20.03.2024','21.03.2024'], 'Medium': ['cpa', 'cpc', 'organic', 'cpa', 'cpc', 'organic', 'direct', 'cpc', 'organic','direct', 'cpc'], 'Conversions': [1, 2, 4, 5, 1, 3, 2, 4, 2, 5, 3]}

df1 = pd.DataFrame(d)

In [5]:
df1.to_sql('conversion', engine, if_exists='replace', index=False)

11

Откроем окно при помощи OVER() и просуммируем столбец c конверсиями:

In [6]:
query = '''SELECT 
      Date, 
      Medium,
      Conversions,
      SUM(Conversions) OVER() AS 'Sum' 
    FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

AttributeError: 'OptionEngine' object has no attribute 'execute'

Инструкция **OVER()** использована без предложений. В таком варианте окном будет весь набор данных и никакая сортировка не применяется. В итоге получаем в новом столбце сумму по всем строкам по полю Conversions.

Теперь применим инструкцию **PARTITION BY**, которая определяет столбец, по которому будет производиться группировка и является ключевой в разделении набора строк на окна:

In [19]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date) AS 'Sum' 
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum
0,18.03.2024,cpa,1,7
1,18.03.2024,cpc,2,7
2,18.03.2024,organic,4,7
3,19.03.2024,cpa,5,15
4,19.03.2024,cpc,1,15
5,19.03.2024,organic,3,15
6,19.03.2024,direct,2,15
7,19.03.2024,cpc,4,15
8,20.03.2024,organic,2,7
9,20.03.2024,direct,5,7


Инструкция PARTITION BY сгруппировала строки по полю «Date». Теперь для каждой группы рассчитывается своя сумма значений столбца «Conversions».

Отсортируем значения внутри окна при помощи **ORDER BY**:

In [20]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date ORDER BY Medium) AS 'Sum'  
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum
0,18.03.2024,cpa,1,1
1,18.03.2024,cpc,2,3
2,18.03.2024,organic,4,7
3,19.03.2024,cpa,5,5
4,19.03.2024,cpc,1,10
5,19.03.2024,cpc,4,10
6,19.03.2024,direct,2,12
7,19.03.2024,organic,3,15
8,20.03.2024,direct,5,5
9,20.03.2024,organic,2,7


К предложению PARTITION BY добавилось ORDER BY по полю «Medium». Таким образом мы указали, что хотим видеть сумму не всех значений в окне, а для каждого значения «Conversions» сумму со всеми предыдущими. То есть мы посчитали нарастающий итог.

### ROWS и RANGE

Инструкция **ROWS** позволяет ограничить строки в окне, указывая фиксированное количество строк, предшествующих или следующих за текущей.

Инструкция **RANGE**, в отличие от **ROWS**, работает не со строками, а с диапазоном строк в инструкции ORDER BY. То есть под одной строкой для RANGE могут пониматься несколько физических строк одинаковых по рангу.

Обе инструкции **ROWS** и **RANGE** всегда используются вместе с **ORDER BY**.

В выражении для ограничения строк ROWS или RANGE также можно использовать следующие ключевые слова:

* UNBOUNDED PRECEDING — указывает, что окно начинается с первой строки группы

* UNBOUNDED FOLLOWING – с помощью данной инструкции можно указать, что окно заканчивается на последней строке группы

* CURRENT ROW – инструкция указывает, что окно начинается или заканчивается на текущей строке

* BETWEEN «граница окна» AND «граница окна» — указывает нижнюю и верхнюю границу окна

* «Значение» PRECEDING – определяет число строк перед текущей строкой (не допускается в предложении RANGE)

* «Значение» FOLLOWING — определяет число строк после текущей строки (не допускается в предложении RANGE)

In [21]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date ORDER BY Conversions ROWS BETWEEN CURRENT ROW AND 1 FOLLOWING) AS 'Sum' 
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum
0,18.03.2024,cpa,1,3
1,18.03.2024,cpc,2,6
2,18.03.2024,organic,4,4
3,19.03.2024,cpc,1,3
4,19.03.2024,direct,2,5
5,19.03.2024,organic,3,7
6,19.03.2024,cpc,4,9
7,19.03.2024,cpa,5,5
8,20.03.2024,organic,2,7
9,20.03.2024,direct,5,5


В данном случае сумма рассчитывается по текущей и следующей ячейке в окне. А последняя строка в окне имеет то же значение, что и столбец «Conversions», потому что больше не с чем складывать.

### Виды оконных функций

Выделяют следующие типы оконных функций:

* Агрегатные функции

* Ранжирующие функции

* Функции смещения

* Аналитические функции

### Агрегатные функции

Агрегатные функции – это функции, которые выполняют на наборе данных арифметические вычисления и возвращают итоговое значение.

* SUM – возвращает сумму значений в столбце;

* COUNT — вычисляет количество значений в столбце (значения NULL не учитываются);

* AVG — определяет среднее значение в столбце;

* MAX — определяет максимальное значение в столбце;

* MIN — определяет минимальное значение в столбце.

In [22]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        SUM(Conversions) OVER(PARTITION BY Date) AS 'Sum', 
        COUNT(Conversions) OVER(PARTITION BY Date) AS 'Count',
        AVG(Conversions) OVER(PARTITION BY Date) AS 'Avg', 
        MAX(Conversions) OVER(PARTITION BY Date) AS 'Max', 
        MIN(Conversions) OVER(PARTITION BY Date) AS 'Min'  
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Sum,Count,Avg,Max,Min
0,18.03.2024,cpa,1,7,3,2.333333,4,1
1,18.03.2024,cpc,2,7,3,2.333333,4,1
2,18.03.2024,organic,4,7,3,2.333333,4,1
3,19.03.2024,cpa,5,15,5,3.000000,5,1
4,19.03.2024,cpc,1,15,5,3.000000,5,1
5,19.03.2024,organic,3,15,5,3.000000,5,1
6,19.03.2024,direct,2,15,5,3.000000,5,1
7,19.03.2024,cpc,4,15,5,3.000000,5,1
8,20.03.2024,organic,2,7,2,3.500000,5,2
9,20.03.2024,direct,5,7,2,3.500000,5,2


### Ранжирующие функции

Ранжирующие функции – это функции, которые ранжируют значение для каждой строки в окне. Например, их можно использовать для того, чтобы присвоить порядковый номер строке или составить рейтинг.

* **ROW_NUMBER** – функция возвращает номер строки и используется для нумерации;

* **RANK** — функция возвращает ранг каждой строки. В данном случае значения уже анализируются и, в случае нахождения одинаковых, возвращает одинаковый ранг с пропуском следующего значения;

* **DENSE_RANK** — функция возвращает ранг каждой строки. Но в отличие от функции RANK, она для одинаковых значений возвращает ранг, не пропуская следующий;

* **NTILE** – это функция, которая позволяет определить к какой группе относится текущая строка. Количество групп задается в скобках.

In [23]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        ROW_NUMBER() OVER(PARTITION BY Date ORDER BY Conversions) AS 'Row_number', 
        RANK() OVER(PARTITION BY Date ORDER BY Conversions) AS 'Rank', 
        DENSE_RANK() OVER(PARTITION BY Date ORDER BY Conversions) AS 'Dense_Rank', 
        NTILE(3) OVER(PARTITION BY Date ORDER BY Conversions) AS 'Ntile' 
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Row_number,Rank,Dense_Rank,Ntile
0,18.03.2024,cpa,1,1,1,1,1
1,18.03.2024,cpc,2,2,2,2,2
2,18.03.2024,organic,4,3,3,3,3
3,19.03.2024,cpc,1,1,1,1,1
4,19.03.2024,direct,2,2,2,2,1
5,19.03.2024,organic,3,3,3,3,2
6,19.03.2024,cpc,4,4,4,4,2
7,19.03.2024,cpa,5,5,5,5,3
8,20.03.2024,organic,2,1,1,1,1
9,20.03.2024,direct,5,2,2,2,2


### Функции смещения

Функции смещения – это функции, которые позволяют перемещаться и обращаться к разным строкам в окне, относительно текущей строки, а также обращаться к значениям в начале или в конце окна.

* **LAG**/**LEAD** – функция LAG обращается к данным из предыдущей строки окна, а LEAD к данным из следующей строки. Функцию можно использовать для того, чтобы сравнивать текущее значение строки с предыдущим или следующим. Имеет три параметра: столбец, значение которого необходимо вернуть, количество строк для смещения (по умолчанию 1), значение, которое необходимо вернуть если после смещения возвращается значение NULL;

* **FIRST_VALUE**/**LAST_VALUE** — с помощью функции можно получить первое/последнее значение в окне. В качестве параметра принимает столбец, значение которого необходимо вернуть.

In [24]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        LAG(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS 'Lag', 
        LEAD(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS 'Lead', 
        FIRST_VALUE(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS 'First_Value', 
        LAST_VALUE(Conversions) OVER(PARTITION BY Date ORDER BY Date) AS 'Last_Value'  
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Lag,Lead,First_Value,Last_Value
0,18.03.2024,cpa,1,NaN,2.0,1,4
1,18.03.2024,cpc,2,1.0,4.0,1,4
2,18.03.2024,organic,4,2.0,NaN,1,4
3,19.03.2024,cpa,5,NaN,1.0,5,4
4,19.03.2024,cpc,1,5.0,3.0,5,4
5,19.03.2024,organic,3,1.0,2.0,5,4
6,19.03.2024,direct,2,3.0,4.0,5,4
7,19.03.2024,cpc,4,2.0,NaN,5,4
8,20.03.2024,organic,2,NaN,5.0,2,5
9,20.03.2024,direct,5,2.0,NaN,2,5


### Аналитические функции

Аналитические функции — это функции которые возвращают информацию о распределении данных и используются для статистического анализа.

* **CUME_DIST** — вычисляет интегральное распределение (относительное положение) значений в окне;

* **PERCENT_RANK** — вычисляет относительный ранг строки в окне;

* **PERCENTILE_CONT** — вычисляет процентиль на основе постоянного распределения значения столбца. В качестве параметра принимает процентиль, который необходимо вычислить (в этой статье я рассказываю как посчитать медиану, благодаря этой функции);

* **PERCENTILE_DISC** — вычисляет определенный процентиль для отсортированных значений в наборе данных. В качестве параметра принимает процентиль, который необходимо вычислить.

In [13]:
query = '''SELECT 
        Date,
        Medium,
        Conversions,
        CUME_DIST() OVER(PARTITION BY Date ORDER BY Conversions) AS 'Cume_Dist',
        PERCENT_RANK() OVER(PARTITION BY Date ORDER BY Conversions) AS 'Percent_Rank'
        FROM conversion
        '''
db = pd.read_sql_query(sql = query, con = engine)
db

,Date,Medium,Conversions,Cume_Dist,Percent_Rank
0,18.03.2024,cpa,1,0.333333,0.00
1,18.03.2024,cpc,2,0.666667,0.50
2,18.03.2024,organic,4,1.000000,1.00
3,19.03.2024,cpc,1,0.200000,0.00
4,19.03.2024,direct,2,0.400000,0.25
5,19.03.2024,organic,3,0.600000,0.50
6,19.03.2024,cpc,4,0.800000,0.75
7,19.03.2024,cpa,5,1.000000,1.00
8,20.03.2024,organic,2,0.500000,0.00
9,20.03.2024,direct,5,1.000000,1.00


Источник: https://thisisdata.ru/blog/uchimsya-primenyat-okonnyye-funktsii/

Дополнительно попрактиковаться с запросами можно в тренажёре: https://www.w3schools.com/sql/

## Практические задания

1. Загрузите данные из датасета: https://www.kaggle.com/datasets/nelgiriyewithana/global-weather-repository

Найти максимальное значение среди средних температур по странам, посчитанных для каждой страны отдельно (т. е. группировка при расчёте средней температуры по странам).

In [2]:
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import sqlite3

In [3]:
weather_df = pd.read_csv('GlobalWeatherRepository.csv')

In [4]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39176 entries, 0 to 39175
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   country                       39176 non-null  object 
 1   location_name                 39176 non-null  object 
 2   latitude                      39176 non-null  float64
 3   longitude                     39176 non-null  float64
 4   timezone                      39176 non-null  object 
 5   last_updated_epoch            39176 non-null  int64  
 6   last_updated                  39176 non-null  object 
 7   temperature_celsius           39176 non-null  float64
 8   temperature_fahrenheit        39176 non-null  float64
 9   condition_text                39176 non-null  object 
 10  wind_mph                      39176 non-null  float64
 11  wind_kph                      39176 non-null  float64
 12  wind_degree                   39176 non-null  int64  
 13  w

In [5]:
weather_df.describe()

,latitude,longitude,last_updated_epoch,temperature_celsius,temperature_fahrenheit,wind_mph,wind_kph,wind_degree,pressure_mb,pressure_in,...,gust_kph,air_quality_Carbon_Monoxide,air_quality_Ozone,air_quality_Nitrogen_dioxide,air_quality_Sulphur_dioxide,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,moon_illumination
count,39176.000000,39176.000000,3.917600e+04,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,...,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.000000,39176.00000
mean,19.300985,21.762872,1.702208e+09,19.128089,66.430457,7.432908,11.964774,162.324893,1013.981009,29.942176,...,19.949086,578.825502,43.346416,14.556509,8.274155,25.508875,46.037579,1.619691,2.471360,50.73032
std,24.522593,65.684612,5.216470e+06,10.747296,19.345272,5.160754,8.307518,106.544531,7.431803,0.219338,...,11.895674,1381.080021,32.462037,27.036714,22.635800,64.094843,107.461981,1.072684,2.695042,34.96374
min,-41.300000,-175.200000,1.693301e+09,-41.900000,-43.400000,2.200000,3.600000,1.000000,958.000000,28.290000,...,0.000000,96.800000,0.000000,0.000000,0.000000,0.500000,0.500000,1.000000,1.000000,0.00000
25%,3.750000,-6.840000,1.697576e+09,12.000000,53.600000,3.800000,6.100000,70.000000,1010.000000,29.830000,...,10.900000,237.000000,18.175000,1.000000,0.500000,2.600000,4.700000,1.000000,1.000000,17.00000
50%,17.250000,23.240000,1.702408e+09,22.000000,71.600000,5.600000,9.000000,150.000000,1013.000000,29.910000,...,17.900000,290.400000,40.800000,4.500000,1.900000,7.600000,13.000000,1.000000,1.000000,51.00000
75%,41.320000,49.880000,1.706722e+09,27.000000,80.600000,10.500000,16.900000,250.000000,1018.000000,30.060000,...,26.300000,447.300000,62.900000,15.650000,6.400000,23.300000,39.800000,2.000000,2.000000,85.00000
max,64.100000,179.220000,1.711039e+09,45.400000,113.700000,91.900000,148.000000,360.000000,1074.000000,31.710000,...,155.200000,41870.102000,555.000000,575.800000,557.000000,1558.800000,3566.400000,6.000000,10.000000,100.00000


In [6]:
weather_df.head(10)

,country,location_name,latitude,longitude,timezone,last_updated_epoch,last_updated,temperature_celsius,temperature_fahrenheit,condition_text,...,air_quality_PM2.5,air_quality_PM10,air_quality_us-epa-index,air_quality_gb-defra-index,sunrise,sunset,moonrise,moonset,moon_phase,moon_illumination
0,Afghanistan,Kabul,34.52,69.18,Asia/Kabul,1693301400,2023-08-29 14:00,28.8,83.8,Sunny,...,7.9,11.1,1,1,05:24 AM,06:24 PM,05:39 PM,02:48 AM,Waxing Gibbous,93
1,Albania,Tirana,41.33,19.82,Europe/Tirane,1693301400,2023-08-29 11:30,27.0,80.6,Partly cloudy,...,28.2,29.6,2,3,06:04 AM,07:19 PM,06:50 PM,03:25 AM,Waxing Gibbous,93
2,Algeria,Algiers,36.76,3.05,Africa/Algiers,1693301400,2023-08-29 10:30,28.0,82.4,Partly cloudy,...,6.4,7.9,1,1,06:16 AM,07:21 PM,06:46 PM,03:50 AM,Waxing Gibbous,93
3,Andorra,Andorra La Vella,42.50,1.52,Europe/Andorra,1693301400,2023-08-29 11:30,10.2,50.4,Sunny,...,0.5,0.8,1,1,07:16 AM,08:34 PM,08:08 PM,04:38 AM,Waxing Gibbous,93
4,Angola,Luanda,-8.84,13.23,Africa/Luanda,1693301400,2023-08-29 10:30,25.0,77.0,Partly cloudy,...,139.6,203.3,4,10,06:11 AM,06:06 PM,04:43 PM,04:41 AM,Waxing Gibbous,93
5,Antigua and Barbuda,Saint John's,17.12,-61.85,America/Antigua,1693301400,2023-08-29 05:30,29.0,84.2,Light rain,...,0.8,1.9,1,1,05:53 AM,06:23 PM,05:36 PM,04:09 AM,Waxing Gibbous,93
6,Argentina,Buenos Aires,-34.59,-58.67,America/Argentina/Buenos_Aires,1693301400,2023-08-29 06:30,9.0,48.2,Clear,...,2.1,3.5,1,1,07:18 AM,06:34 PM,04:57 PM,06:32 AM,Waxing Gibbous,93
7,Armenia,Yerevan,40.18,44.51,Asia/Yerevan,1693301400,2023-08-29 13:30,31.0,87.8,Partly cloudy,...,5.0,6.2,1,1,06:26 AM,07:39 PM,07:05 PM,03:45 AM,Waxing Gibbous,93
8,Australia,Canberra,-35.28,149.22,Australia/Sydney,1693301400,2023-08-29 19:30,13.0,55.4,Clear,...,4.0,5.8,1,1,06:27 AM,05:42 PM,03:19 PM,05:17 AM,Waxing Gibbous,93
9,Austria,Vienna,48.20,16.37,Europe/Vienna,1693301400,2023-08-29 11:30,16.0,60.8,Light rain,...,13.1,14.9,1,2,06:07 AM,07:43 PM,07:27 PM,03:11 AM,Waxing Gibbous,93


In [7]:
conn = sqlite3.connect(r'C:\Users\ivs18\Jupyter Notebook/GlobalWeatherRepository.db')

OperationalError: unable to open database file

In [44]:
conn = sqlite3.connect(':memory:')

In [45]:
def create_connection(path):
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [46]:
connection = create_connection(r'C:\Users\ivs18\Jupyter Notebook/GlobalWeatherRepository.db')

Connection to SQLite DB successful


In [42]:
weather_df.to_sql('Weather', connection, if_exists='replace', index=False)

39176

In [53]:
#Найти максимальное значение среди средних температур по странам, посчитанных для каждой страны отдельно 
#(т. е. группировка при расчёте средней температуры по странам).

query = ''' WITH avg_temp_country AS (SELECT country, AVG(temperature_celsius) AS avg_temp FROM Weather  
           GROUP BY country)
           SELECT country, MAX(avg_temp) AS max_avg_temp FROM avg_temp_country
        '''
db = pd.read_sql_query(sql = query, con = connection)
db

,country,max_avg_temp
0,Paraguay,35.574627


2. Напишите SQL-запрос, который выбирает страны, города в которых не попадают в топ-10 по наиболее высокой температуре.

In [75]:
query = ''' WITH Top_10_Countries AS (SELECT country, temperature_celsius,
            RANK() OVER(ORDER BY temperature_celsius DESC) AS Rank FROM Weather)
           SELECT country FROM Weather 
           WHERE country NOT IN (SELECT country FROM Top_10_Countries WHERE Rank <= 10)
        '''
db = pd.read_sql_query(sql = query, con = connection)
db

,country
0,Afghanistan
1,Albania
2,Algeria
3,Andorra
4,Angola
...,...
38568,Venezuela
38569,Vietnam
38570,Yemen
38571,Zambia


3. Для каждой из стран вычислите разницу средней температуры по стране и средней температуры по всем странам. Помимо этого, вычислите на сколько процентов средняя температура по стране отличается от средней температуры по всем странам. 

In [67]:
query = ''' WITH Avg_Temp_Country AS (SELECT country, AVG(temperature_celsius) AS avg_temp_country FROM Weather  
           GROUP BY country), Total_Avg_Temp AS (SELECT AVG(temperature_celsius) AS total_avg_temp FROM Weather)
              SELECT country, (avg_temp_country - total_avg_temp) AS diff_temp, ROUND(((avg_temp_country - total_avg_temp)/total_avg_temp) * 100, 2) AS percent_diff 
              FROM Avg_Temp_Country, Total_Avg_Temp  
        '''
db = pd.read_sql_query(sql = query, con = connection)
db

,country,diff_temp,percent_diff
0,Afghanistan,-13.434556,-70.23
1,Albania,-5.934556,-31.03
2,Algeria,-1.223114,-6.39
3,Andorra,-16.084308,-84.09
4,Angola,7.438081,38.89
...,...,...,...
180,Venezuela,11.317683,59.17
181,Vietnam,1.649026,8.62
182,Yemen,-2.303711,-12.04
183,Zambia,3.421663,17.89


4. Выполните задание описанное в https://disk.yandex.ru/d/0vK-0DyMVZsBLA

In [76]:
from sqlalchemy import create_engine, text as sql_text
engine = create_engine('sqlite:///SQL_lesson_5.db', echo = False)

emp_data = {'tab_num': list(range(1,4)), 'FIO': list('FIO{}'.format(i) for i in range(1,4))}
df_emp_data = pd.DataFrame(emp_data)
df_emp_data.to_sql('emp_data', engine, if_exists='replace', index=False)
query = 'SELECT * FROM emp_data'
db = pd.read_sql_query(sql = sql_text(query), con = engine.connect())
db

emp_marks = {'tab_num': [1, 1, 1, 2, 2], 'mark': [72, 91, 86, 93, 89]}
df_emp_marks = pd.DataFrame(emp_marks)
df_emp_marks.to_sql('emp_marks', engine, if_exists='replace', index=False)
query = 'SELECT * FROM emp_marks'
db = pd.read_sql_query(sql = sql_text(query), con = engine.connect())
db

sessions_duration = {'sess_id': list(range(1,6)), 'duration': [30, 199, 299, 580, 1000]}
df_sessions_duration = pd.DataFrame(sessions_duration)
df_sessions_duration.to_sql('sessions_duration', engine, if_exists='replace', index=False)
query = 'SELECT * FROM sessions_duration'
db = pd.read_sql_query(sql = sql_text(query), con = engine.connect())

In [81]:
query = ''' SELECT tab_num, AVG(mark) FROM emp_marks 
            GROUP BY tab_num
        '''
db = pd.read_sql_query(sql = query, con = engine.connect())
db

,tab_num,AVG(mark)
0,1,83.0
1,2,91.0


In [84]:
query = ''' WITH avg_marks AS (SELECT tab_num, AVG(mark) AS avg_mrk FROM emp_marks 
            GROUP BY tab_num) 
            SELECT tab_num, avg_mrk FROM avg_marks WHERE (avg_mrk >= 90)
        '''
db = pd.read_sql_query(sql = query, con = engine.connect())
db

,tab_num,avg_mrk
0,2,91.0


In [110]:
query = ''' 
            SELECT tab_num, mark, DENSE_RANK() 
            OVER(ORDER BY mark DESC) AS rank FROM emp_marks
        '''
db = pd.read_sql_query(sql = query, con = engine.connect())
db

,tab_num,mark,rank
0,2,93,1
1,1,91,2
2,2,89,3
3,1,86,4
4,1,72,5


In [111]:
query = ''' WITH Avg_Marks AS (SELECT tab_num, AVG(mark) AS emp_avg_marks FROM emp_marks 
            GROUP BY tab_num)
            SELECT tab_num, FIO, emp_avg_marks FROM Avg_Marks, emp_data
            LEFT JOIN emp_data ON emp_data.tab_num = emp_marks.tab_num
        '''
db = pd.read_sql_query(sql = query, con = engine.connect())
db

OperationalError: (sqlite3.OperationalError) ambiguous column name: tab_num
[SQL:  WITH Avg_Marks AS (SELECT tab_num, AVG(mark) AS emp_avg_marks FROM emp_marks 
            GROUP BY tab_num)
            SELECT tab_num, FIO, emp_avg_marks FROM Avg_Marks, emp_data
            LEFT JOIN emp_data ON emp_data.tab_num = emp_marks.tab_num
        ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [114]:
query = '''
           SELECT MIN(duration) AS min_duration, AVG(duration) AS avg_duration, MAX(duration) AS max_duration,
           PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY duration) OVER(PARTITION BY sess_id) AS percentile_50, 
           PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY duration) OVER(PARTITION BY sess_id) AS percentile_75,
           PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY duration) OVER(PARTITION BY sess_id) AS percentile_99 
           FROM sessions_duration;
        '''
db = pd.read_sql_query(sql = query, con = engine.connect())
db

OperationalError: (sqlite3.OperationalError) near "(": syntax error
[SQL: 
           SELECT MIN(duration) AS min_duration, AVG(duration) AS avg_duration, MAX(duration) AS max_duration,
           PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY duration) OVER(PARTITION BY sess_id) AS percentile_50, 
           PERCENTILE_CONT(0.75) WITHIN GROUP (ORDER BY duration) OVER(PARTITION BY sess_id) AS percentile_75,
           PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY duration) OVER(PARTITION BY sess_id) AS percentile_99 
           FROM sessions_duration;
        ]
(Background on this error at: https://sqlalche.me/e/14/e3q8)